# 1. Tratamento dos dados e criacao da coluna target

## Importar bibliotecas

In [1]:
# ! pip install xlrd
# ! pip install openpyxl
# ! pip install pydantic
# ! pip install joblib

In [2]:
import numpy as np

import joblib

In [3]:
def carregar_df_unico():
    df = joblib.load("../out/database_unico.joblib")
    print("arquivo joblib carregado com sucesso")
    return df


In [30]:
def select_columns_for_calc(df):
    
    colunas_para_calculos = [
        "Ativo Total", 
        "Ativo Realizável a Longo Prazo",
        "Ativo Circulante",
        "Passivo Circulante", 
        "Passivo Não Circulante", 
        "Patrimônio Líquido", 
        "Receita Líquida de Vendas e/ou Serviços", 
        "Lucro/Prejuízo do Período", 
        "Adj Close",
        "IBOV"]

    columns_replace={
        "Ativo Total": "ativo_total", 
        "Ativo Circulante": "ativo_circ",
        "Ativo Realizável a Longo Prazo": "ativo_permanente",
        "Passivo Circulante": "passivo_circ", 
        "Passivo Não Circulante": "passivo_nao_circ", 
        "Patrimônio Líquido": "patrimonio_liq", 
        "Receita Líquida de Vendas e/ou Serviços": "receita_liq", 
        "Lucro/Prejuízo do Período": "lucro_op", 
        "Adj Close": "cotacao"}

    df_c = df[colunas_para_calculos]
    df_c = df_c.rename(columns=columns_replace)
    df_c.iloc[:,:-1] = df_c.iloc[:,:-1].astype(float)

    print("colunas para calculo selecionadas com sucesso")

    return df_c


In [16]:
def calcular_dupont(df_in):
    
    df = df_in.copy()

    af = df["ativo_total"] / df["patrimonio_liq"]
    ra = df["lucro_op"] / df["ativo_total"]
    ga = df["receita_liq"] / df["ativo_total"]
    ml = df["lucro_op"] / df["receita_liq"]
    rpl = af * ra
    roe = df["lucro_op"] / df["patrimonio_liq"]
    roa = ga * ml

    df["RPL"] = rpl
    df["ROE"] = roe
    df["ROA"] = roa
    df["AF"] = af
    df["RA"] = ra
    df["GA"] = ga
    df["ML"] = ml

    print("indicadores dupont criados com sucesso")

    return df


In [6]:
def calcular_indicadores_financeiros(df_in):

    df = df_in.copy()

    # Indicadores de endividamento
    part_cap_terceiros = (df["passivo_circ"] + df["passivo_nao_circ"]) / df["patrimonio_liq"]
    comp_endividamento = df["passivo_circ"] / (df["passivo_circ"] + df["passivo_nao_circ"])
    imob_patrimonio_liq = df["ativo_permanente"] / df["patrimonio_liq"]
    imob_recursos_n_corr = df["ativo_permanente"] / (df["patrimonio_liq"] + df["passivo_nao_circ"])

    df["part_cap_terceiros"] = part_cap_terceiros
    df["comp_endividamento"] = comp_endividamento
    df["imob_patrimonio_liq"] = imob_patrimonio_liq
    df["imob_recursos_n_corr"] = imob_recursos_n_corr

    # Indicadores de liquidez

    liq_corrente = df["ativo_circ"] / df["passivo_circ"]

    df["liq_corrente"] = liq_corrente

    print("indicadores financeiros criados com sucesso")

    return df



In [33]:
def remover_colunas_de_calculo(df):
    colunas_remover = ['ativo_total', 'ativo_permanente', 'ativo_circ', 'passivo_circ',
       'passivo_nao_circ', 'patrimonio_liq', 'receita_liq', 'lucro_op']
    
    df = df.drop(colunas_remover, axis=1)
    
    print("Colunas para calculo removidas com sucesso")

    return df

In [ ]:
def transformar_valores_em_porcentagem_delta(df):
    pass

## Criacao de rotulos - coluna target

## Criar coluna resultado baseado em cotacao e IBOV

- cotacao e Ibov, considerar valores do proximo trimestre 
- Considerar comprar (1): variacao da cotacao acima da variacao do IBOV
- Considerar Vender (0): variacao da cotacao de 2% abaixo da variacao do IBOV

In [7]:
def criar_coluna_decisao(df_in): 
    
    df = df_in.copy()
    
    df = df.sort_index()

    df["cotacao_var"] = df["cotacao"].shift(-1) / df["cotacao"] - 1
    df["IBOV_var"] = df["IBOV"].shift(-1) / df["IBOV"] - 1
    df["resultado"] = df["cotacao_var"] - df["IBOV_var"]

    condicoes = [
        (df["resultado"] > 0), 
        (df["resultado"] < -0.02)
    ]
    valores = [1, 0]

    df["decisao"] = np.select(condicoes, valores)

    df = df.drop(["cotacao", "cotacao_var", "IBOV", "IBOV_var", "resultado"], axis=1)

    print("coluna decisao/target criada com sucesso")

    return df


## Salvar dataframe em joblib

In [8]:
def salvar_df_in_joblib(df, file_name: str = "../out/tratamento_target.joblib"):

    joblib.dump(df, file_name)

    print("arquivo joblib de df decisao criado com sucesso")


# Execucao

In [34]:
def main():

    df = carregar_df_unico()

    df = select_columns_for_calc(df)

    df = calcular_dupont(df)

    df = calcular_indicadores_financeiros(df)

    df = criar_coluna_decisao(df)

    df = remover_colunas_de_calculo(df)

    df = transformar_valores_em_porcentagem_delta(df)

    # # salvar em arquivo joblib
    # salvar_df_in_joblib(bd)
    
    return df
    
df = main()

arquivo joblib carregado com sucesso
colunas para calculo selecionadas com sucesso
indicadores dupont criados com sucesso
indicadores financeiros criados com sucesso
coluna decisao/target criada com sucesso
Colunas para calculo removidas com sucesso


In [35]:
df

,RPL,ROE,ROA,AF,RA,GA,ML,part_cap_terceiros,comp_endividamento,imob_patrimonio_liq,imob_recursos_n_corr,liq_corrente,decisao
0,0.698183,0.698183,0.017592,39.687980,0.017592,0.439573,0.040020,36.844341,0.323647,4.857637,0.187410,1.441260,0
1,4.340236,4.340236,0.043348,100.126226,0.043348,0.513462,0.084422,93.016335,0.328613,13.060862,0.205845,1.370436,0
2,0.132940,0.132940,-0.003925,-33.866506,-0.003925,0.387054,-0.010142,-33.310219,0.357605,-4.708834,0.230844,1.153807,0
3,0.043497,0.043497,0.000852,51.040747,0.000852,0.450332,0.001892,48.169037,0.356261,6.355157,0.198547,1.358459,1
4,0.288335,0.288335,0.003195,90.254630,0.003195,0.405689,0.007875,78.036877,0.298557,9.443841,0.169431,1.790702,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723,0.015544,0.015544,0.009724,1.598470,0.009724,0.126738,0.076727,0.598470,0.513472,0.130997,0.101456,1.134020,0
1724,0.046428,0.046428,0.028638,1.621214,0.028638,0.130235,0.219894,0.621214,0.516697,0.129933,0.099930,1.157492,1
1725,0.000000,0.000000,0.000000,1.639099,0.000000,0.120508,0.000000,0.639099,0.527846,0.130003,0.099868,1.259057,1
1726,0.021109,0.021109,0.012950,1.630022,0.012950,0.120281,0.107667,0.630022,0.505335,0.151138,0.115228,1.249345,0
